In [1]:
import pandas as pd
import numpy as np
import gzip
import matplotlib.pyplot as plt
import seaborn as sns

# CARGA DE DATOS

* Carga del json comprimido

In [2]:
steam_games = pd.read_json('../Datasets/steam_games.json.gz', compression='gzip', lines=True)

In [3]:
# Creo copia del pd
steam_games_origin = steam_games.copy()

* Verifico Nulos

In [4]:
steam_games.isna().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

* Elimino todas las filas completamente nulas, representan un 70% de los datos

In [5]:
steam_games = steam_games.dropna(how='all')

In [6]:
steam_games.isna().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

* Se encuentran 2 registros sin 'publisher', 'app_name' y 'title', procedo a borrarlos.

In [7]:
steam_games[steam_games.app_name.isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
90890,None,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",None,0.0,317160.0,None


In [8]:
# Elimino los registros que tienen nombres vacios y no se pueden estimar.

indices_a_borrar = steam_games[steam_games.app_name.isna()].index
steam_games.drop(index=indices_a_borrar, inplace=True)

In [9]:
steam_games.isna().sum()

publisher       8050
genres          3282
app_name           0
title           2048
url                0
release_date    2066
tags             162
reviews_url        1
specs            669
price           1376
early_access       0
id                 1
developer       3297
dtype: int64

- Columna 'title' y 'app_name' contienen los nombres de los juegos, por lo tanto copleto los faltantes de una con los faltantes de la otra.

In [10]:
steam_games['title'] = steam_games['app_name'].fillna(steam_games['title'])

- Completo los generos vacios con las etiquetas de los Tags

In [11]:
steam_games['genres'] = steam_games['tags'].fillna(steam_games['genres'])

In [12]:
# Lleno las Tags con los generos (Quedan 138 valores nulos en ambas columnas)
steam_games['tags'] = steam_games['genres'].fillna(steam_games['tags'])
# Estos resultados no tienen genre, tags, publisher, specs, developer, se procesede a eliminar los registros.
indices_drop_2 = steam_games[steam_games['tags'].isna() & steam_games['specs'].isna()].index
steam_games.drop(index=indices_drop_2, inplace=True)

In [13]:
steam_games.isna().sum()

publisher       8043
genres           131
app_name           0
title              0
url                0
release_date    2066
tags             131
reviews_url        1
specs            662
price           1372
early_access       0
id                 1
developer       3290
dtype: int64

- En las url figuran los numeros de id, completo el id nulo con ese valor 


In [14]:
steam_games[steam_games['id'].isna()]['url'].values[0]

'http://store.steampowered.com/app/200260'

In [15]:
import re

url = steam_games[steam_games['id'].isna()]['url'].values[0]

#Definir la expresión regular
expresion_regular = r"/app/(\d+)"

#Buscar el número en la URL
resultado = re.search(expresion_regular, url)

#Verificar si se encontró el patrón y extraer el número
if resultado:
    numero = resultado.group(1)
    print("Número extraído:", numero)
else:
    print("No se encontró el número en la URL.")

Número extraído: 200260


In [16]:
def input_id (url):
    #Definir la expresión regular
    expresion_regular = r"/app/(\d+)"

    #Buscar el número en la URL
    resultado = re.search(expresion_regular, url)

    #Verificar si se encontró el patrón y extraer el número
    if resultado:
        numero = resultado.group(1)
        return int(numero)
    else:
        return None 

In [17]:
steam_games['url'].apply(input_id)

88310     761140
88311     643980
88312     670290
88313     767400
88314     773570
           ...  
120440    773640
120441    733530
120442    610660
120443    658870
120444    681550
Name: url, Length: 32126, dtype: int64

In [18]:
# verifico los id Nulos
steam_games[steam_games['id'].isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Open World, Batman, Adventure, Stealt...",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [19]:
# Se coloca el id extraido de la url en los nulos de la columna id
steam_games.loc[steam_games['id'].isna(), 'id'] = steam_games.loc[steam_games['id'].isna(), 'url'].apply(input_id)

In [20]:
# Paso el id a entero
steam_games['id'] = steam_games['id'].astype('int') 

In [21]:
steam_games['id'].isna().sum()

0

- Eliminación de duplicados

In [22]:
steam_games.drop_duplicates(subset=['publisher', 'app_name', 'title','release_date','price', 'early_access', 'id','developer'],keep='last', inplace=True)

- Eliminación de columnas que no voy dar uso

In [23]:
columnas_drop = ['url', 'reviews_url']
steam_games.drop(columnas_drop, axis=1, inplace=True)

In [24]:
# verificación de nulos actualizada
steam_games.isna().sum()

publisher       8043
genres           131
app_name           0
title              0
release_date    2066
tags             131
specs            662
price           1372
early_access       0
id                 0
developer       3290
dtype: int64

- Completo datos publisher y developer entre si 

In [25]:

steam_games['developer'] = steam_games['publisher'].fillna(steam_games['developer'])

In [26]:
#Creación categoria 'Otros' en publisher y developer que tengan valores nulos (3225 registros)
steam_games['publisher'].fillna('otros', inplace=True)
steam_games['developer'].fillna('otros', inplace=True)

In [27]:
# Aplico los cambios
steam_games['developer'] = steam_games['developer'].str.capitalize()
steam_games['publisher'] = steam_games['publisher'].str.capitalize()


In [28]:
# Eliminamos espacios en blanco
steam_games[['developer' , 'publisher']] = steam_games[['developer', 'publisher']].apply(lambda x: x.str.strip())

In [29]:
# Verificación de Nulos actuaizada
steam_games.isna().sum()

publisher          0
genres           131
app_name           0
title              0
release_date    2066
tags             131
specs            662
price           1372
early_access       0
id                 0
developer          0
dtype: int64

- Columna *'release_date'*, cambio a formato Date los datos y coloco datos nulos en aquellos que tienen formato incorrecto

In [30]:
steam_games['release_date'] = pd.to_datetime(steam_games['release_date'], format='%Y-%m-%d', errors='coerce')

In [31]:
steam_games

,publisher,genres,app_name,title,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140,Kotoshiro
88311,"Making fun, inc.","[Free to Play, Strategy, Indie, RPG, Card Game...",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980,"Making fun, inc."
88312,Poolians.com,"[Free to Play, Simulation, Sports, Casual, Ind...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400,彼岸领域
88314,Otros,"[Action, Indie, Casual, Sports]",Log Challenge,Log Challenge,NaT,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,0.0,773570,Otros
...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_rus games,"[Strategy, Indie, Casual, Simulation]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,0.0,773640,Ghost_rus games
120441,Sacada,"[Strategy, Indie, Casual]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530,Sacada
120442,Laush studio,"[Indie, Simulation, Racing]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660,Laush studio
120443,Sixnails,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870,Sixnails


In [32]:
# Creo columna 'release_date_unix' la cual contiene los valores de tiempo Unix correspondientes a las fechas de lanzamiento en la columna original 'release_date'. 

steam_games['release_date_unix'] = steam_games['release_date'].apply(lambda x: int(x.timestamp()) if not pd.isna(x) else None)

- Ante la presencia de 2351 valores nulos en la columna 'release_date' y considerando su importancia para el modelo futuro, <br>
 se toma la decisión de utilizar el algoritmo de imputación KNN (K-Nearest Neighbors) <br>
 para estimar y llenar estos valores faltantes de manera más precisa.

In [33]:
# Utilizo el MODELO KNN para estimar valores faltantes y luego imputarlos en la columna 'release_date_unix'

from sklearn.impute import KNNImputer
imputador_knn = KNNImputer(n_neighbors=5)
steam_games['release_date_unix'] = imputador_knn.fit_transform(steam_games[['release_date_unix']])

In [34]:
# Imputo valores faltantes en 'release_date' con valores estimados en columna 'release_date_unix'
steam_games['release_date'] = steam_games['release_date'].fillna(steam_games['release_date_unix'])

In [35]:
# Verificación nulos actualizada
steam_games['release_date'].isna().sum()

0

In [36]:
steam_games['release_date'] = pd.to_datetime(steam_games['release_date_unix'], unit='s')

In [37]:
# Se procede a extraer únicamente el año de la columna 'release_date' y lo guardo en una nueva columna 'year'
steam_games['year']=steam_games['release_date'].dt.year

In [38]:
steam_games.isna().sum()

publisher               0
genres                131
app_name                0
title                   0
release_date            0
tags                  131
specs                 662
price                1372
early_access            0
id                      0
developer               0
release_date_unix       0
year                    0
dtype: int64

In [39]:
# Se eliminan las filas que contienen valores nulos.
# Se identifican y eliminan las columnas que no aportan relevancia al modelo.

# Código para la limpieza de datos:
steam_games_cleaned = steam_games.dropna()  
columnas_no_relevantes = ['publisher', 'title', 'tags', 'specs', 'price', 'early_access', 'developer', 'release_date_unix', 'release_date']  
steam_games_cleaned = steam_games_cleaned.drop(columnas_no_relevantes, axis=1)


In [40]:
steam_games_cleaned.to_csv('../Datasets/steam_games_cleaned.csv',index=False)

with gzip.open('../Datasets/steam_games_cleaned.csv.gz', 'wb') as f:
    steam_games_cleaned.to_csv(f, index=False, encoding='utf-8')

In [41]:
steam_games_cleaned

,genres,app_name,id,year
88310,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,761140,2018
88311,"[Free to Play, Strategy, Indie, RPG, Card Game...",Ironbound,643980,2018
88312,"[Free to Play, Simulation, Sports, Casual, Ind...",Real Pool 3D - Poolians,670290,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,767400,2017
88314,"[Action, Indie, Casual, Sports]",Log Challenge,773570,2015
...,...,...,...,...
120440,"[Strategy, Indie, Casual, Simulation]",Colony On Mars,773640,2018
120441,"[Strategy, Indie, Casual]",LOGistICAL: South Africa,733530,2018
120442,"[Indie, Simulation, Racing]",Russian Roads,610660,2018
120443,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",EXIT 2 - Directions,658870,2017
